In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torchsummary import summary

In [2]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

])

In [3]:
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform = transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform = transform)

100%|██████████| 170M/170M [00:03<00:00, 48.5MB/s]


In [4]:
batch_size = 128
num_class = 10
epochs = 5
learning_rate = 0.001
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [5]:
# Load a pretrained vgg16 model
model = models.vgg16(pretrained=True)

# Freeze the feature layers (optional, but often beneficial initially)
for param in model.features.parameters():
    param.requires_grad = False

# Modify the classifier for CIFAR-10 (10 classes)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 10)





/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 186MB/s]


In [6]:
print(model.classifier)

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=10, bias=True)
)


In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
summary(model, (3,224, 224))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [8]:
optimizer = optim.Adam(model.classifier[6].parameters(), lr=learning_rate)

citerion = nn.CrossEntropyLoss()

def train_model(model, train_loader, epochs):
  model.train()
  for epoch in range(epochs):
    total_running_loss = 0
    correct = 0
    for input, label in train_loader:
      input, label = input.to(device), label.to(device)
      optimizer.zero_grad()
      output = model(input)
      loss = citerion(output, label)
      loss.backward()
      optimizer.step()
      total_running_loss += loss.item() * input.size(0)
      pred = output.argmax(dim=1)
      correct += pred.eq(label).sum().item()
    avg_loss = total_running_loss / len(train_loader.dataset)
    accuracy = correct / len(train_loader.dataset)
    print(f'Epoch: [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}')


train_model(model, train_loader, epochs)

Epoch: [1/5], Loss: 0.6551, Accuracy: 0.7703
Epoch: [2/5], Loss: 0.5620, Accuracy: 0.8022
Epoch: [3/5], Loss: 0.5487, Accuracy: 0.8087
Epoch: [4/5], Loss: 0.5445, Accuracy: 0.8094
Epoch: [5/5], Loss: 0.5514, Accuracy: 0.8080


In [9]:
def evaluate_model (model, test_loader):
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for input, label in test_loader:
      input, label = input.to(device), label.to(device)
      output = model(input)
      loss = citerion(output, label)
      test_loss += loss.item() * input.size(0)
      pred = output.argmax(dim=1)
      correct += pred.eq(label).sum().item()
    avg_loss = test_loss / len(test_loader.dataset)
    accuracy  = correct / len(test_loader.dataset)
    print(f'Test Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}')

evaluate_model(model, test_loader)

Test Loss: 0.4711, Accuracy: 0.8334
